<a href="https://colab.research.google.com/github/skfkeh/runAutoEmployToNotionDB/blob/main/notionForJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from notion_database.search import Search
from notion_database.const.query import Direction, Timestamp

NOTION_API_KEY = os.getenv("NOTION_API_KEY")

def list_databases(notion_api_key):
    S = Search(integrations_token=notion_api_key)
    S.search_database(query="",
                      sort={"direction": Direction.descending,
                            "timestamp": Timestamp.last_edited_time},
                      root_only=False)
    return S.result

db = list_databases(NOTION_API_KEY)
DATABASE_ID = "auto_selected_database_id"
## Notion Database 리스트
for i in db:
    #  print(i['title'][0]['plain_text'] + " : " + i['id'])
     if (i['title'][0]['plain_text'].find("지원 내역") != -1):
          DATABASE_ID = i['id']

# print(DATABASE_ID)

In [ ]:
from notion_database.database import Database
# from pprint import pprint

def get_database_properties(notion_api_key, database_id):
    D = Database(integrations_token=notion_api_key)
    D.retrieve_database(database_id=database_id)
    return D.result

res = get_database_properties(NOTION_API_KEY, DATABASE_ID)
# pprint(res)

In [ ]:
from datetime import datetime, timedelta
import requests, numpy
from bs4 import BeautifulSoup

### 잡알리오 표준직무(NCS) 구분값 정리
def getDetailCode(detailCode):
    code_map = {
        '사업관리': 'R600001',
        '경영': 'R600002',
        '회계': 'R600002',
        '사무': 'R600002',
        '금융': 'R600003',
        '보험': 'R600003',
        '교육': 'R600004',
        '자연': 'R600004',
        '사회과학': 'R600004',
        '보건': 'R600006',
        '의료': 'R600006',
        '사회복지': 'R600007',
        '종교': 'R600007',
        '전기': 'R600019',
        '전자': 'R600019',
        '정보통신': 'R600020'
    }
    return code_map.get(detailCode, "잘못된 연산자입니다.")

def JobInfo(detail_code, region):
    ### 고정값
    s_date = (datetime.now() - timedelta(90)).strftime("%Y.%m.%d")
    e_date = datetime.now().strftime("%Y.%m.%d")
    ing = 2
    pageSet = 50          ## 항목 표시 개수

    ### 유동값
    pageNo = 1

    ### 하이퍼 파라미터
    detail_code = detail_code

    url = f"https://job.alio.go.kr/recruit.do?pageNo={pageNo}&idx=&s_date={s_date}&e_date={e_date}&detail_code={detail_code}&org_type=&org_name=&ing={ing}&search_type=&keyword=&order=REG_DATE&sort=DESC&pageSet={pageSet}"
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    response.encoding = 'utf-8' # Changed encoding from 'cp949' to 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')

    # 검색 결과 영역에서 주식 링크 찾기 (종목 코드 포함된 링크 찾기)
    targetTable = soup.select_one('#frm > table > tbody')
    targetTr = targetTable.select('tr')

    li_employID = []       # 공고 id값
    li_employURL = []      # 공고 id값
    li_employCompany = []  # 채용 기업
    li_employTitle = []    # 공고명
    li_employRegion = []   # 지역
    li_employDate = []     # 마감일
    li_employCategory = [] # (비)정규직
    li_employType = []     # 경력여부

    # targetTr = targetTr[:5]
    for i in targetTr:
        company = i.select_one('td:nth-of-type(4)').text.strip()
        jobRegion = i.select_one('td:nth-of-type(5)').text.strip().replace('\r','').replace('\n','').replace('\t','')
        if ((region != "") and (jobRegion.find(region) == -1)):
            # print("지역 이슈로 [" + company + "] PASS !!\t지역: " + jobRegion)
            continue

        employID = i.select_one('td:nth-of-type(3)').find('a').get('id')[5:]
        company = i.select_one('td:nth-of-type(4)').text.strip()
        title = i.select_one('td:nth-of-type(3)').text.strip()
        period = "20" + i.select_one('td:nth-of-type(8)').text.strip().split(chr(10))[0].replace("\r", "").replace(".", "-")
        # category = i.select_one('td:nth-of-type(6)').text.replace("	", "").replace(chr(10), "").replace(chr(13), "").strip()

        ### 채용구분값 가져오기
        jobDescription = f"https://job.alio.go.kr/recruitview.do?idx={employID}"
        subResponse = requests.get(jobDescription, headers=headers)
        subResponse.encoding = 'utf-8' # Changed encoding from 'cp949' to 'utf-8'
        subSoup = BeautifulSoup(subResponse.text, 'html.parser')
        targetSubTable = subSoup.select_one('#txt > div.detailTxt > table')

        jobType = targetSubTable.select('tr')[1].select('td')[1].text.strip() # 신입 / 경력
        jobType = '경력무관' if (jobType == '신입+경력') else jobType

        jobURL = 'X' if (len(subSoup.select_one('#txt > div.topInfo > p > a').text.strip()) < 5) else subSoup.select_one('#txt > div.topInfo > p > a').text.strip()
        if jobURL == 'X':
          continue;

        li_employCompany.append(company)
        li_employTitle.append(title)
        li_employRegion.append(jobRegion)
        li_employDate.append(period)
        li_employType.append(jobType)
        li_employURL.append(jobURL)
        # li_employCategory.append(category)

    result_arr = numpy.stack([li_employCompany
                         , li_employType
                         , li_employDate
                         , li_employRegion
                         , li_employTitle
                         , li_employURL], axis=1)
    return result_arr

# result = JobInfo(getDetailCode('정보통신'), "서울")
# print(result)


In [ ]:
import requests
from notion_database.page import Page
from notion_database.properties import Properties
from notion_database.database import Database
from datetime import datetime
import re

# ---------- 1) Notion API로 직접 검색 ----------
def find_existing_page_via_api(notion_api_key, database_id, company, apply_deadline_date):
    """
    apply_deadline_date는 'YYYY-MM-DD' 형태의 문자열을 넘겨주세요.
    Notion의 date equals 필터는 날짜 문자열(예: '2025-11-16')로 비교할 수 있습니다.
    """
    url = f"https://api.notion.com/v1/databases/{database_id}/query"
    headers = {
        "Authorization": f"Bearer {notion_api_key}",
        "Notion-Version": "2022-06-28",  # 또는 사용중인 버전
        "Content-Type": "application/json"
    }

    payload = {
        "filter": {
            "and": [
                {
                    "property": "회사명",
                    "title": {
                        "equals": company
                    }
                },
                {
                    "property": "서류 마감일",
                    "date": {
                        "equals": apply_deadline_date
                    }
                }
            ]
        },
        "page_size": 1
    }

    resp = requests.post(url, headers=headers, json=payload)
    resp.raise_for_status()
    data = resp.json()
    if data.get("results"):
        return data["results"][0]
    return None

# ---------- 2) DB 구조 조회(기존 코드 재사용용) ----------
def get_database_properties(notion_api_key, database_id):
    D = Database(integrations_token=notion_api_key)
    D.retrieve_database(database_id=database_id)
    return D.result

# ---------- 3) 페이지 생성 (중복 체크 포함) ----------
def create_database_page(notion_api_key, database_id, properties, child_properties=False):
    # 1) 중복 검사에 사용할 값(회사명, 날짜) 전처리
    company = properties.get("회사명")
    raw_deadline = properties.get("서류 마감일")

    # raw_deadline이 "YYYY-MM-DD HH:MM:SS+0900" 처럼 들어오면 'YYYY-MM-DD'만 추출
    apply_deadline_date = None
    if raw_deadline:
        # 가능한 형식들을 처리 (날짜 문자열에서 YYYY-MM-DD 추출)
        # 예: "2025-11-16 12:00:00+0900" 또는 "2025-11-16"
        m = re.search(r"(\d{4}-\d{2}-\d{2})", str(raw_deadline))
        if m:
            apply_deadline_date = m.group(1)

    # 회사명 또는 날짜가 없으면 중복검사 생략(원하면 여기서 다르게 처리 가능)
    if company and apply_deadline_date:
        existing = find_existing_page_via_api(notion_api_key, database_id, company, apply_deadline_date)
        if existing:
            print(f"❌ 이미 존재하는 페이지 발견. 생성하지 않습니다. 회사명={company}, 서류 마감일={apply_deadline_date}")
            return existing

    # 2) 없으면 Properties 채우고 페이지 생성
    PROPERTY = Properties()
    res = get_database_properties(notion_api_key, database_id)

    for k, v in res['properties'].items():
        property_type = v['type']
        if k in properties:
            value = properties[k]
            # property_type이 'status'일 경우 이전에 건너뛰던 로직 유지
            if property_type == 'status':
                continue
            elif property_type == 'people' and not value:
                continue
            elif property_type == 'date' and not value:
                continue
            else:
                # 주의: Properties 클래스의 set_* 메서드가 해당 타입을 지원해야 함
                getattr(PROPERTY, f'set_{property_type}')(k, value)

    P = Page(integrations_token=notion_api_key)
    P.create_page(database_id=database_id, properties=PROPERTY)
    print(f"✅ 새로운 페이지 생성됨: {company}, {raw_deadline}")
    return P.result

In [ ]:
def create_page_with_link(api_key, db_id, company_name, link):
    url = "https://api.notion.com/v1/pages"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Notion-Version": "2022-06-28",
        "Content-Type": "application/json"
    }

    payload = {
        "parent": {"database_id": db_id},
        "properties": {
            "회사명": {
                "title": [
                    {
                        "text": {
                            "content": company_name,
                            "link": {"url": link}
                        }
                    }
                ]
            }
        }
    }

    res = requests.post(url, json=payload, headers=headers)
    return res.json()


In [ ]:
from datetime import datetime, timezone, timedelta
import time

def get_now_kst():
    kst = timezone(timedelta(hours=9))
    return datetime.now(tz=kst).isoformat()

def main():
    startTime = time.time() ## 시작시점
    print("!! 채용 공고 긁어오기 !!")
    position = "정보통신"
    result_seoul = JobInfo(getDetailCode(position), "서울")

    for i in result_seoul:
        properties = {
            # '회사명' : create_page_with_link(
            #             NOTION_API_KEY,
            #             DATABASE_ID,
            #             i[0],
            #             i[4]
            #         ),
            '회사명' : i[0],
            '경력여부' : i[1],
            '직군' : position,
            '결과' : '대기 중',
            # 추천: date 비교 용도로는 'YYYY-MM-DD' 형식으로 저장/전달하세요.
            '서류 마감일' : i[2], #datetime.now().strftime("%Y-%m-%d"),
            # '서류 마감일': get_now_kst(),
            '서류 발표일' : '',
            '필기/과제 마감일' : '',
            '필기결과일' : '',
            '1차면접일' : '',
            '2차면접일' : '',
            'Status' : '할 일',
            '담당자' : '',
            '비고' : i[5]
        }

        create_database_page(NOTION_API_KEY, DATABASE_ID, properties)

    result_gyeonggi = JobInfo(getDetailCode(position), "경기")

    for i in result_gyeonggi:
        properties = {
            # '회사명' : create_page_with_link(
            #             NOTION_API_KEY,
            #             DATABASE_ID,
            #             i[0],
            #             i[4]
            #         ),
            '회사명' : i[0],
            '경력여부' : i[1],
            '직군' : position,
            '결과' : '대기 중',
            # 추천: date 비교 용도로는 'YYYY-MM-DD' 형식으로 저장/전달하세요.
            '서류 마감일' : i[2], #datetime.now().strftime("%Y-%m-%d"),
            # '서류 마감일': get_now_kst(),
            '서류 발표일' : '',
            '필기/과제 마감일' : '',
            '필기결과일' : '',
            '1차면접일' : '',
            '2차면접일' : '',
            'Status' : '할 일',
            '담당자' : '',
            '비고' : i[5]
        }

        create_database_page(NOTION_API_KEY, DATABASE_ID, properties)

    endTime = time.time() ## 시작시점
    print(f"\n프로그램이 종료되었습니다.\n{int((endTime - startTime)/60)} 분 {int((endTime - startTime)%60)} 초 ({endTime - startTime:.5f} sec) 소요")

if __name__ == "__main__":
	  main()